# תוכן עניינים
- [אודות Guidance](../../../../code/01.Introduce)
- [הגדרות](../../../../code/01.Introduce)
- [יצירה ללא מגבלות](../../../../code/01.Introduce)
- [דיבור עבור Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [בחירה](../../../../code/01.Introduce)
- [שרשרת מחשבה](../../../../code/01.Introduce)
- [יצירת JSON](../../../../code/01.Introduce)
- [יצירת HTML](../../../../code/01.Introduce)


# על Guidance
Guidance היא ספריית Python בקוד פתוח מוכחת לשליטה בתוצרים של כל מודל שפה (LM). באמצעות קריאה אחת ל-API, ניתן לבטא (ב-Python) את המגבלה התוכנית המדויקת שהמודל חייב לעמוד בה וליצור את הפלט המובנה ב-JSON, Python, HTML, SQL או כל מבנה שהשימוש דורש.

Guidance שונה מטכניקות הנחיה מסורתיות. היא אוכפת מגבלות על ידי הכוונת המודל, טוקן אחר טוקן, בשכבת ההסקה, מה שמוביל לתוצרים איכותיים יותר ומפחית עלויות וזמן תגובה בעד 30–50% כאשר משתמשים בה בתרחישים מובנים מאוד.

למידע נוסף על Guidance, בקרו ב-[מאגר הציבורי ב-GitHub](https://github.com/guidance-ai/guidance) או צפו ב-[Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) בכנס Microsoft Build.


# התקנה
1. התקן את Guidance באמצעות `pip install guidance --pre`
2. פרוס נקודת קצה של Phi 3.5 mini ב-Azure על ידי מעבר לכתובת https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml ולחיצה על כפתור "Deploy"
3. שמור את מפתח ה-API של נקודת הקצה שלך במשתנה סביבה בשם `AZURE_PHI3_KEY` ואת כתובת ה-URL במשתנה סביבה בשם `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# יצירה ללא מגבלות
ניתן ליצור טקסט ללא מגבלות באמצעות הפונקציה `gen()`. זה זהה לשימוש במודל ללא Guidance.

## עיצוב שיחה
כמו רבים ממודלי השיחה, Phi-3 מצפה להודעות בין משתמש לעוזר בפורמט מסוים. Guidance תומך בתבנית השיחה של Phi-3 וינהל את עיצוב השיחה עבורך. כדי ליצור סבבי שיחה, יש לשים כל חלק מהשיחה בתוך בלוק `with user()` או `with assistant()`. ניתן להשתמש בבלוק `with system()` כדי להגדיר את הודעת המערכת.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## חיסכון בטוקנים
בתרחישים מובנים מאוד, Guidance יכול לדלג על טוקנים וליצור רק את הטוקנים הנחוצים, מה שמשפר ביצועים, מגדיל יעילות וחוסך בעלויות API. טוקנים שנוצרו מוצגים במחברת זו עם רקע מודגש. טוקנים מאולצים מוצגים ללא הדגשה ועולים אותו הדבר כמו טוקנים קלט, שמוערכים בכשליש מעלות טוקנים פלט.

*הערה:* הדוגמה הראשונה עם יצירה ללא מגבלות לא הצליחה לאלץ אף טוקן מכיוון שלא סיפקנו מגבלות.


# מדברים על Phi 3  
עם Guidance, ניתן בקלות להכניס טקסט לתגובות של המודל. זה יכול להיות מועיל אם רוצים לכוון את הפלט של המודל לכיוון מסוים.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# הגבלה באמצעות רג'קס  
בדוגמה הקודמת, Phi 3 סיפקה הסברים נוספים לאחר שענתה על השאלה עם `Canberra`. כדי להגביל את הפלט של המודל למילה אחת בלבד, ניתן להשתמש ברג'קס.  


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

עם ה-regex, רק המילה `Canberra` נוצרת.


# בחירה מתוך מספר אפשרויות
כאשר ידועות כמה אפשרויות אפשריות, ניתן להשתמש בפונקציה `select()` כדי לגרום למודל לבחור מתוך רשימת אפשרויות.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

עם `select()`, רק הטוקן `Can` נוצר. מכיוון ש-`Canberra` היא האפשרות היחידה שיכולה להשלים את התגובה, הטוקנים הנותרים הוכרחו.


# שרשרת מחשבה
שרשרת מחשבה היא טכניקה שיכולה לעזור לשפר את איכות הפלט של המודל על ידי עידודו לעבד בעיה שלב אחר שלב. בדרך כלל, כדי להגיע לתשובה סופית, נדרשים מספר סבבי הנחיה. ראשית, יש להנחות את המודל לחשוב שלב אחר שלב. לאחר מכן, יש להנחות את המודל שוב כדי לספק את התשובה הסופית. עם ממשקי API סטנדרטיים להסקת שיחה, זה דורש 2 קריאות API, ו"שרשרת המחשבה" שהמודל יוצר מחויבת פעמיים – פעם אחת כטוקנים פלט כשהמודל יצר אותה, ופעם נוספת כטוקנים קלט עבור הקריאה השנייה. עם Guidance, כל התהליך הרב-שלבי מעובד ומחויב כחלק מקריאת API אחת, מה שמפחית עלויות וזמן תגובה.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# יצירת JSON
ניתן להשתמש ב-Guidance כדי להבטיח יצירה של JSON שתואם לסכמת JSON או מודל pydantic, כמו סכמת פרופיל המשתמש שמוצגת כאן.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## יצירת HTML

ניתן להשתמש ב-Guidance ליצירת קוד ולעמידה בדרישות הסינטקטיות של שפת התכנות. בחלק זה, ניצור תוכנית Guidance קטנה לכתיבת דפי HTML פשוטים מאוד.

נחלק את דף האינטרנט לחלקים קטנים יותר, כאשר לכל חלק תהיה פונקציית Guidance משלו. לאחר מכן, נשלב את החלקים בפונקציה הסופית שלנו כדי ליצור דף HTML.  
לבסוף, נריץ את הפונקציה הזו מול מודל המופעל על ידי Guidance ב-Azure AI.

*הערה:* זה לא הולך להיות יוצר HTML מלא; המטרה היא להראות כיצד ניתן ליצור פלט מובנה לצרכים האישיים שלך.

נתחיל בייבוא מה שנדרש מ-Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

דפי אינטרנט ב-HTML הם בעלי מבנה מאוד מסודר, ואנו נשתמש בהנחיות כדי 'לכפות' את החלקים הללו של הדף.  
כאשר אנו דורשים באופן מפורש טקסט מהמודל, עלינו לוודא שהוא אינו כולל שום דבר שיכול להיחשב כתג - כלומר, עלינו להימנע מהשימוש בתווים '<' ו-'>'.  


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

אנחנו יכולים להשתמש בפונקציה הזו כדי ליצור טקסט בתוך תגית HTML שרירותית:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

עכשיו, בואו ניצור את כותרת העמוד.  
כחלק מזה, אנחנו צריכים ליצור כותרת לעמוד:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

גוף דף ה-HTML עומד להתמלא בכותרות ופסקאות.  
אנחנו יכולים להגדיר פונקציה שתעשה כל אחד מהם:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

עכשיו, הפונקציה שמגדירה את גוף ה-HTML עצמו.  
זה משתמש ב-`select()` עם `recurse=True` כדי ליצור מספר כותרות ופסקאות:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

כעת, אנו מגיעים לפונקציה שמייצרת את דף ה-HTML המלא.  
אנו מוסיפים את תג הפתיחה של HTML, לאחר מכן יוצרים את הכותרת, לאחר מכן את הגוף, ולבסוף מצרפים את תג הסיום של HTML:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

אנו מספקים מעטפת ידידותית למשתמש, שתאפשר לנו:
- להגדיר את הטמפרטורה של ההפקה
- ללכוד את הדף שנוצר מאובייקט ה-Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

לאחר מכן נוכל לכתוב את הפלט לקובץ:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

ולראות את התוצאה](./sample_page.html).



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
